In [2]:
!pip install selenium webdriver-manager pandas requests pypdf tabula-py
!pip install gspread
!pip install google-auth

In [22]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service 
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import TimeoutException
import time
import pandas as pd
import os
import requests
import io
import re
from pypdf import PdfReader 
import tabula 
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import locale
import gspread
from google.oauth2.service_account import Credentials

DATOS_CARTERA_ESQUEMA = [
    {"Accion": "Pampa Holding S.A. (PAMP-AR)", "Porcentaje": "0%"},
]
N_FILAS_ESPERADAS = 10
DATOS_CARTERA_MOCK = DATOS_CARTERA_ESQUEMA

NOMBRE_FONDO_EXTRAIDO = "Nombre no extraído"
SOCIEDAD_GERENTE = "Santander AM"
GOOGLE_SHEETS_CREDENTIALS = r"C:\Users\Rolon\OneDrive\Escritorio\TUI-DATA-DEV\TUI\Trabajo Final\Proyecto WEB\proyecto-final-tui-060f98c3047c.json" 
GOOGLE_SHEET_ID = '1W2NLc-uUAseoZ4o9A_8DRuE6h4zmbhSjgFnH8_BCwM8' 
SHEET_NAME = 'Datos Mensuales' 

URL_INICIAL = "https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12"

pdf_url = None
original_window = None

try:
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options) 
    driver.maximize_window()
    driver.implicitly_wait(10)

    original_window = driver.current_window_handle
    
    print(f"Abriendo la URL: {URL_INICIAL}")
    driver.get(URL_INICIAL)
    
    time.sleep(5) 
    
    XPATH_ENLACE = "//a[contains(., 'Reporte mensual - CAFCI')]" 
    enlace_cartera = driver.find_element(By.XPATH, XPATH_ENLACE)
    
    driver.execute_script("arguments[0].scrollIntoView(true);", enlace_cartera)
    driver.execute_script("arguments[0].click();", enlace_cartera)
    
    WebDriverWait(driver, 10).until(EC.number_of_windows_to_be(2))
    
    for window_handle in driver.window_handles:
        if window_handle != original_window:
            driver.switch_to.window(window_handle)
            break
            
    time.sleep(3) 
    pdf_url = driver.current_url 
    
    if pdf_url and pdf_url.endswith(".pdf"):
        pass
    else:
        pdf_url = None 

    if pdf_url:
        driver.close()
        driver.switch_to.window(original_window)

except Exception as e:
    print(f"❌ Error CRÍTICO en la navegación/clic: {e}")

finally:
    if 'driver' in locals() and hasattr(driver, 'service') and driver.service.process:
        driver.quit() 

try:
    try:
        locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8') 
    except locale.Error:
        try:
            locale.setlocale(locale.LC_TIME, 'Spanish_Spain')
        except locale.Error:
            locale.setlocale(locale.LC_TIME, 'es')
except locale.Error:
    pass


periodo_extraido_str = "Fecha No encontrada" 
fecha_procesada = None 
df_composicion_final = pd.DataFrame()

if pdf_url and pdf_url.endswith(".pdf"):
    try:
        
        response = requests.get(pdf_url)
        pdf_file = io.BytesIO(response.content)

        reader = PdfReader(pdf_file)
        pdf_file.seek(0) 
        full_text = reader.pages[0].extract_text() 
        
        nombre_pattern = re.compile(r'Superfondo\s+(.*?)\s*-\s*Clase\s+\w', re.IGNORECASE | re.DOTALL)
        nombre_match = nombre_pattern.search(full_text)

        if nombre_match:
            NOMBRE_FONDO_EXTRAIDO = "Superfondo " + nombre_match.group(1).strip() + " - Clase A" 
            print(f"✅ Nombre del fondo extraído: {NOMBRE_FONDO_EXTRAIDO}")
        else:
            NOMBRE_FONDO_EXTRAIDO = "Superfondo Renta Variable - Clase A"
            print(f"⚠️ Nombre del fondo no encontrado. Usando valor predeterminado: {NOMBRE_FONDO_EXTRAIDO}")
        
        date_pattern = re.compile(r'Datos\s*al\s*(\d{1,2}.*?\d{4})', re.IGNORECASE | re.DOTALL)
        match = date_pattern.search(full_text)
        
        if match:
            fecha_bruta = match.group(1).strip()
            periodo_extraido_str = re.sub(r'\s+', ' ', fecha_bruta) 
            try:
                fecha_procesada = pd.to_datetime(periodo_extraido_str, format="%d de %B %Y")
            except Exception:
                fecha_procesada = periodo_extraido_str
        else:
            fecha_procesada = periodo_extraido_str

        print("\n🔍 Intentando extraer TODA la tabla de composición con 'tabula-py'...")

        TABLA_AREA = [380, 300, 640, 770] 

        pdf_file.seek(0)
        
        dfs_from_pdf = tabula.read_pdf(
            pdf_file, 
            pages=1, 
            multiple_tables=False, 
            output_format="dataframe",
            area=TABLA_AREA,       
            stream=True,           
            encoding='latin-1' 
        )
        
        if dfs_from_pdf and not dfs_from_pdf[0].empty:
            df_pdf_data = dfs_from_pdf[0] 
            
            df_pdf_data.dropna(how='all', inplace=True)
            
            col_acciones = df_pdf_data.columns[0]
            acciones_dinamicas = df_pdf_data[col_acciones].dropna().head(N_FILAS_ESPERADAS).tolist()

            cols_porcentajes = [col for col in df_pdf_data.columns if 'Unnamed' in col or '%' in str(df_pdf_data[col].iloc[0])]
            
            porcentajes_dinamicos = (
                df_pdf_data[cols_porcentajes]
                .stack() 
                .dropna() 
                .head(N_FILAS_ESPERADAS) 
                .tolist()
            )
            
            if len(acciones_dinamicas) == N_FILAS_ESPERADAS and len(porcentajes_dinamicos) == N_FILAS_ESPERADAS:
                
                df_composicion_final = pd.DataFrame({
                    'Accion': acciones_dinamicas,
                    'Porcentaje': porcentajes_dinamicos
                })
                
                print(f"✅ Extracción 100% DINÁMICA completada con {N_FILAS_ESPERADAS} filas.")
            else:
                df_composicion_final = pd.DataFrame(DATOS_CARTERA_MOCK)
                print(f"⚠️ Alerta: Fallo en alineación. Usando el mock anterior.")
                
        else:
            df_composicion_final = pd.DataFrame(DATOS_CARTERA_MOCK)
            print("❌ No se pudo extraer la tabla del PDF. Usando mock anterior.")
            
    except Exception as e:
        df_composicion_final = pd.DataFrame(DATOS_CARTERA_MOCK)
        print(f"❌ Error CRÍTICO durante la descarga/lectura/procesamiento: {e}")

if not df_composicion_final.empty:
    try:
        print("\n--- INICIANDO PROCESAMIENTO Y GUARDADO EN GOOGLE SHEETS ---")
        
        df_nuevo = df_composicion_final.copy()
        
        df_nuevo['Porcentaje'] = df_nuevo['Porcentaje'].astype(str).str.replace('%', '', regex=False)
        df_nuevo['Porcentaje'] = pd.to_numeric(df_nuevo['Porcentaje'], errors='coerce') / 100
        
        df_nuevo['Periodo'] = fecha_procesada
        
        if pd.api.types.is_datetime64_any_dtype(df_nuevo['Periodo']):
            df_nuevo['Periodo'] = df_nuevo['Periodo'].dt.strftime('%Y-%m-%d')
        else:
            df_nuevo['Periodo'] = df_nuevo['Periodo'].astype(str)
            
        df_nuevo['Nombre_Fondo'] = NOMBRE_FONDO_EXTRAIDO
        df_nuevo['Sociedad_Gerente'] = SOCIEDAD_GERENTE
        df_nuevo = df_nuevo[['Periodo', 'Nombre_Fondo', 'Sociedad_Gerente', 'Accion', 'Porcentaje']]
        
        scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
        credentials = Credentials.from_service_account_file(GOOGLE_SHEETS_CREDENTIALS, scopes=scope)
        gc = gspread.authorize(credentials)

        sh = gc.open_by_key(GOOGLE_SHEET_ID)
        try:
            worksheet = sh.worksheet(SHEET_NAME)
        except gspread.WorksheetNotFound:
            print(f"⚠️ Pestaña '{SHEET_NAME}' no encontrada. Usando la primera pestaña (index 0).")
            worksheet = sh.get_worksheet(0)
            
        data_anterior = worksheet.get_all_records()
        df_anterior = pd.DataFrame(data_anterior)
        
        if not df_anterior.empty and 'Periodo' in df_anterior.columns:
            df_anterior = df_anterior[df_nuevo.columns.tolist()]
            
            df_combinado = pd.concat([df_anterior, df_nuevo], ignore_index=True)
            df_combinado['Periodo'] = df_combinado['Periodo'].astype(str)
            df_combinado.drop_duplicates(subset=['Periodo', 'Accion'], keep='last', inplace=True)
            
            filas_totales = len(df_combinado)
        else:
            df_combinado = df_nuevo
            filas_totales = len(df_nuevo)
            
        worksheet.clear()
        worksheet.update([df_combinado.columns.values.tolist()] + df_combinado.values.tolist(), value_input_option='USER_ENTERED')

        print(f"🚀 Proceso completado. Datos anexados a Google Sheets con éxito. Filas totales: {filas_totales}")

    except Exception as e:
        print(f"❌ Error al conectar o guardar en Google")

Abriendo la URL: https://www.santander.com.ar/empresas/inversiones/informacion-fondos#/detail/12
✅ Nombre del fondo extraído: Superfondo Renta Variable - Clase A

🔍 Intentando extraer TODA la tabla de composición con 'tabula-py'...
✅ Extracción 100% DINÁMICA completada con 10 filas.

--- INICIANDO PROCESAMIENTO Y GUARDADO EN GOOGLE SHEETS ---
🚀 Proceso completado. Datos anexados a Google Sheets con éxito. Filas totales: 90
